In [117]:
# Instalar spaCy y descargar el modelo de inglés
!pip install spacy
!python -m spacy download en_core_web_sm

# Instalar SentenceTransformers para embeddings semánticos
!pip install sentence-transformers

# Instalar NLTK para manejo de sinónimos
!pip install nltk

# Descargar recursos de NLTK
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sgsr_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sgsr_\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [375]:
import pandas as pd

# Nombre del archivo CSV generado previamente
archivo_csv = 'SDG_final.csv'

# Definir el goal_id fijo
goal_id_fijo = '17'  # Cambia este valor según el GOAL que desees (por ejemplo, '2', '3', etc.)

try:
    # Cargar el archivo CSV
    df = pd.read_csv(archivo_csv, delimiter=',', encoding='utf-8')
    print("Archivo CSV cargado exitosamente.\n")
    
    # Verificar que las columnas necesarias existen en el DataFrame
    columnas_necesarias = {'goal_id', 'target_id', 'texto'}
    if not columnas_necesarias.issubset(df.columns):
        missing = columnas_necesarias - set(df.columns)
        raise KeyError(f"Faltan las siguientes columnas en el archivo CSV: {missing}")
    
    # Filtrar las filas que corresponden al goal_id especificado
    df_goal = df[df['goal_id'].astype(str) == str(goal_id_fijo)]
    
    # Verificar si se encontraron filas para el goal_id especificado
    if df_goal.empty:
        print(f"No se encontraron registros para el goal_id '{goal_id_fijo}'.")
    else:
        # Extraer la columna 'texto' y almacenarla en una lista llamada 'targets'
        targets = df_goal['texto'].tolist()
        goal_ids = df_goal['goal_id'].tolist()
        target_ids = df_goal['target_id'].tolist()
        
        # Mostrar la cantidad de textos extraídos
        print(f"Se han extraído {len(targets)} textos para el goal_id {goal_id_fijo}.\n")
        
        # Mostrar los textos extraídos
        print("Lista 'targets':")
        for idx, texto in enumerate(targets, start=1):
            print(f"{idx}. {texto}\n")
        
        # (Opcional) Guardar la lista 'targets' en un archivo de texto
        # Descomenta las siguientes líneas si deseas guardar los textos en un archivo
        """
        nombre_archivo_salida = f'targets_goal_{goal_id_fijo}.txt'
        with open(nombre_archivo_salida, 'w', encoding='utf-8') as f:
            for texto in targets:
                f.write(texto + '\n')
        print(f"Lista 'targets' guardada en el archivo '{nombre_archivo_salida}'.")
        """

except FileNotFoundError:
    print(f"Error: El archivo '{archivo_csv}' no se encontró en el directorio actual.")
except pd.errors.EmptyDataError:
    print(f"Error: El archivo '{archivo_csv}' está vacío.")
except pd.errors.ParserError:
    print(f"Error: El archivo '{archivo_csv}' no pudo ser parseado. Verifica el delimitador y el formato del archivo.")
except KeyError as e:
    print(f"Error de clave: {e}")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Archivo CSV cargado exitosamente.

Se han extraído 19 textos para el goal_id 17.

Lista 'targets':
1. Strengthen domestic resource mobilization, including through international support to developing countries, to improve domestic capacity for tax and other revenue collection | Total government revenue as a proportion of GDP, by source | Proportion of domestic budget funded by domestic taxes

2. Developed countries to implement fully their official development assistance commitments, including the commitment by many developed countries to achieve the target of 0.7 per cent of ODA/GNI to developing countries and 0.15 to 0.20 per cent of ODA/GNI to least developed countries; ODA providers are encouraged to consider setting a target to provide at least 0.20 per cent of ODA/GNI to least developed countries | Net official development assistance, total and to least developed countries, as a proportion of the Organization for Economic Cooperation and Development (OECD) Development Assistance C

In [376]:
import pandas as pd

# Nombre del archivo CSV
archivo_csv = 'Compiled-Keywords-for-SDG.csv'

# Definir el parámetro fijo X (por ejemplo, 'SDG 1')
X = f'SDG {goal_id_fijo}'  # Cambia este valor según el GOAL que desees (e.g., 'SDG 2', 'SDG3', etc.)

try:
    # Cargar el archivo CSV
    df = pd.read_csv(archivo_csv, delimiter=',', encoding='utf-8')
    print(f"Archivo '{archivo_csv}' cargado exitosamente.\n")
    
    # Verificar que las columnas X y 'Misc' existen en el DataFrame
    columnas_necesarias = [X, 'Misc']
    columnas_presentes = df.columns.tolist()
    
    for columna in columnas_necesarias:
        if columna not in columnas_presentes:
            raise KeyError(f"La columna '{columna}' no existe en el archivo CSV. Columnas disponibles: {columnas_presentes}")
    
    # Extraer las columnas X y 'Misc', eliminando valores nulos y espacios en blanco
    elementos_X = df[X].dropna().astype(str).str.strip()
    elementos_Misc = df['Misc'].dropna().astype(str).str.strip()
    
    # Combinar ambas series
    combinados = pd.concat([elementos_X, elementos_Misc])
    
    # Eliminar entradas vacías después de stripping y convertir a lista de elementos únicos
    combinados = combinados[combinados != '']
    palabras_clave = combinados.unique().tolist()
    
    # Eliminar posibles duplicados adicionales y valores vacíos
    palabras_clave = [elemento for elemento in palabras_clave if elemento]
    
    # Mostrar la lista 'palabras_clave'
    print(f"Lista 'palabras_clave' para la columna '{X}' y 'Misc':\n")
    for idx, elemento in enumerate(palabras_clave, start=1):
        print(f"{idx}. {elemento}")
    
    # (Opcional) Guardar la lista 'palabras_clave' en un archivo de texto
    # Descomenta las siguientes líneas si deseas guardar los resultados
    """
    nombre_archivo_salida = f'palabras_clave_{X.replace(" ", "_")}_y_Misc.txt'
    with open(nombre_archivo_salida, 'w', encoding='utf-8') as f:
        for elemento in palabras_clave:
            f.write(elemento + '\n')
    print(f"\nLa lista 'palabras_clave' ha sido guardada en el archivo '{nombre_archivo_salida}'.")
    """

except FileNotFoundError:
    print(f"Error: El archivo '{archivo_csv}' no se encontró en el directorio actual.")
except pd.errors.EmptyDataError:
    print(f"Error: El archivo '{archivo_csv}' está vacío.")
except pd.errors.ParserError:
    print(f"Error: El archivo '{archivo_csv}' no pudo ser parseado. Verifica el delimitador y el formato del archivo.")
except KeyError as e:
    print(f"Error de clave: {e}")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Archivo 'Compiled-Keywords-for-SDG.csv' cargado exitosamente.

Lista 'palabras_clave' para la columna 'SDG 17' y 'Misc':

1. Capacity building
2. Civil society partnerships
3. Communication technologies
4. Debt sustainability
5. Development assistance
6. Disaggregated data
7. Doha Development Agenda
8. Entrepreneurship
9. Environmentally sound technologies
10. Foreign direct investments
11. Fostering innovation
12. Free trade
13. Fundamental principles of official statistics
14. Global partnership
15. Global partnership for sustainable development
16. Global stability
17. International aid
18. International cooperation
19. International population and housing census
20. International support
21. International support for developing countries
22. Knowledge sharing
23. Multi-stakeholder partnerships
24. Poverty eradication
25. Public-private partnerships
26. Science cooperation agreements
27. Technology cooperation agreements
28. Technology transfer
29. Weighted tariff average
30. Women 

In [377]:
import re
import unicodedata

def normalizar_texto(texto):
    """
    Normaliza el texto:
    - Convierte a minúsculas
    - Elimina acentos
    - Elimina caracteres especiales
    - Elimina espacios en blanco excesivos
    """
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar acentos
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    # Eliminar caracteres especiales
    texto = re.sub(r'[^a-z0-9\s]', '', texto)
    # Eliminar espacios en blanco excesivos
    texto = ' '.join(texto.split())
    return texto

# Normalizar palabras clave
palabras_clave_normalizadas = [normalizar_texto(palabra) for palabra in palabras_clave]

# Normalizar targets
targets_normalizados = [normalizar_texto(target) for target in targets]

print("Palabras clave normalizadas:")
print(palabras_clave_normalizadas)
print("\nPrimer target normalizado:")
print(targets_normalizados[0])


Palabras clave normalizadas:
['capacity building', 'civil society partnerships', 'communication technologies', 'debt sustainability', 'development assistance', 'disaggregated data', 'doha development agenda', 'entrepreneurship', 'environmentally sound technologies', 'foreign direct investments', 'fostering innovation', 'free trade', 'fundamental principles of official statistics', 'global partnership', 'global partnership for sustainable development', 'global stability', 'international aid', 'international cooperation', 'international population and housing census', 'international support', 'international support for developing countries', 'knowledge sharing', 'multistakeholder partnerships', 'poverty eradication', 'publicprivate partnerships', 'science cooperation agreements', 'technology cooperation agreements', 'technology transfer', 'weighted tariff average', 'women entrepreneurs', 'world trade organization', 'accountability', 'alternative energy', 'biodiversity', 'caring for count

In [378]:
import spacy
from tqdm import tqdm

# Cargar el modelo de spaCy para inglés
# Nota: Asegúrate de haber descargado el modelo 'en_core_web_sm' previamente
# Puedes hacerlo ejecutando: python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

def extraer_conceptos(target):
    """
    Extrae conceptos clave de un target utilizando spaCy.
    """
    doc = nlp(target)
    conceptos = set()
    for chunk in doc.noun_chunks:
        # Filtrar chunks demasiado cortos
        if len(chunk.text.split()) < 2:
            continue
        # Filtrar chunks que sean solo stop words
        if all(token.is_stop for token in chunk):
            continue
        # Añadir el chunk normalizado
        conceptos.add(normalizar_texto(chunk.text))
    return conceptos

# Extraer conceptos de todos los targets
targets_conceptos = []
for target in tqdm(targets_normalizados, desc="Extrayendo conceptos"):
    conceptos = extraer_conceptos(target)
    targets_conceptos.append(conceptos)

# Ejemplo de conceptos extraídos para el primer target
print("\nConceptos extraídos para el primer target:")
print(targets_conceptos[0])


Extrayendo conceptos: 100%|██████████| 19/19 [00:00<00:00, 105.55it/s]


Conceptos extraídos para el primer target:
{'domestic taxes', 'source proportion', 'domestic capacity', 'developing countries', 'other revenue collection total government revenue', 'a proportion', 'domestic budget', 'domestic resource mobilization', 'international support'}


In [379]:
from nltk.corpus import wordnet as wn
import nltk

# Descargar recursos de NLTK (si no se han descargado previamente)
nltk.download('wordnet')
nltk.download('omw-1.4')

def obtener_sinonimos(palabra):
    """
    Obtiene sinónimos de una palabra en inglés utilizando WordNet.
    """
    sinonimos = set()
    for syn in wn.synsets(palabra, lang='eng'):
        for lemma in syn.lemmas('eng'):
            # Reemplazar guiones bajos con espacios y normalizar
            sinonimos.add(normalizar_texto(lemma.name().replace('_', ' ')))
    return list(sinonimos)

# Expansión de palabras clave
palabras_clave_expandidas = set(palabras_clave_normalizadas)
for palabra in palabras_clave_normalizadas:
    # Dividir en palabras para obtener sinónimos de cada palabra clave compuesta
    palabras = palabra.split()
    for p in palabras:
        sinonimos = obtener_sinonimos(p)
        palabras_clave_expandidas.update(sinonimos)

palabras_clave_expandidas = list(palabras_clave_expandidas)

print("\nPalabras clave expandidad con sinónimos:")
print(palabras_clave_expandidas)



Palabras clave expandidad con sinónimos:
['broken', 'channelize', 'median', 'respectable', 'build up', 'little', 'tourism', 'home', 'irregular migration', 'conveyance', 'downcast', 'progress', 'voice', 'cosmos', 'manage', 'h2o', 'investiture', 'target', 'defend', 'reassign', 'abide', 'raptus', 'equal', 'maverick', 'foreign', 'caring', 'pocketsize', 'intergenerational', 'displacement', 'low impact agriculture', 'put up', 'sore', 'pull', 'shipping', 'keister', 'equal rights to economic resources', 'organisation', 'development', 'menage', 'agendum', 'data', 'body politic', 'relinquish', 'equalize', 'give care', 'recycle', 'social', 'partake in', 'diminished', 'goal', 'development assistance', 'swap', 'doe', 'rival', 'caring for country', 'backside', 'affirm', 'posterior', 'civil', 'lowpitched', 'good governance', 'irrigate', 'energy', 'environmentally sound technologies', 'craft', 'unrivalled', 'modal', 'well', 'responsibleness', 'firm', 'reuse', 'tolerate', 'take chances', 'piss', 'comm

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sgsr_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sgsr_\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [380]:
from sentence_transformers import SentenceTransformer, util
import torch

# Cargar el modelo de embeddings multilingüe
modelo = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Puedes elegir otro modelo si lo prefieres

# Generar embeddings para palabras clave expandidas
embeddings_palabras = modelo.encode(palabras_clave_expandidas, convert_to_tensor=True)

# Generar embeddings para conceptos únicos
# Primero, aplanamos la lista de conceptos
conceptos_unicos = list(set(concepto for conceptos in targets_conceptos for concepto in conceptos))
embeddings_conceptos = modelo.encode(conceptos_unicos, convert_to_tensor=True)

print(f"\nTotal de conceptos únicos: {len(conceptos_unicos)}")
print(f"Total de palabras clave expandidas: {len(palabras_clave_expandidas)}")



Total de conceptos únicos: 139
Total de palabras clave expandidas: 1031


In [381]:
# Calculamos la matriz de similitud
similitudes = util.pytorch_cos_sim(embeddings_conceptos, embeddings_palabras)

# Definir umbral de similitud
umbral_similitud = 0.7

# Crear un diccionario para mapear conceptos a palabras clave
mapa_concepto_palabra = {}

for idx, concepto in enumerate(conceptos_unicos):
    similitudes_concepto = similitudes[idx]
    indices_similares = torch.where(similitudes_concepto >= umbral_similitud)[0]
    palabras_similares = [palabras_clave_expandidas[i] for i in indices_similares]
    if palabras_similares:
        mapa_concepto_palabra[concepto] = palabras_similares

# Ejemplo de mapeo
print("\nEjemplo de mapeo de concepto a palabras clave:")
for concepto, palabras in list(mapa_concepto_palabra.items())[:5]:
    print(f"- {concepto}: {palabras}")



Ejemplo de mapeo de concepto a palabras clave:
- the organization: ['organisation', 'organization']
- the world trade organization: ['world trade organization']
- developing countries proportion: ['developing countries']
- coordinated policies: ['policy']
- many developed countries: ['developing countries', 'least developed countries']


In [382]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Extraer los embeddings de las palabras clave
embeddings_palabras_np = embeddings_palabras.cpu().numpy()

# Aplicar Agglomerative Clustering con el métrico 'cosine'
# Nota: 'cosine' métrico se utiliza directamente, no 'precomputed'
# linkage='average' es compatible con 'cosine' métrico
cluster = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=0.5,  # Ajusta este umbral según tus necesidades
    linkage='average',
    metric='cosine'
    #metric=None
)

clusters = cluster.fit_predict(embeddings_palabras_np)

# Agrupar palabras clave por clusters
from collections import defaultdict

clusters_dict = defaultdict(list)
for palabra, c in zip(palabras_clave_expandidas, clusters):
    clusters_dict[c].append(palabra)

print("\nClusters de palabras clave similares:")
for c, palabras in clusters_dict.items():
    if len(palabras) > 1:
        print(f"Cluster {c}: {palabras}")



Clusters de palabras clave similares:
Cluster 158: ['broken', 'break']
Cluster 8: ['channelize', 'transmit', 'channel', 'communicating', 'communication', 'channelise', 'communication technologies']
Cluster 222: ['respectable', 'honest', 'polite']
Cluster 117: ['build up', 'development', 'develop', 'construct', 'creation', 'build', 'developed', 'developing', 'founding', 'invention']
Cluster 125: ['little', 'minor', 'smallscale', 'small']
Cluster 232: ['tourism', 'eco tourism', 'touristry']
Cluster 259: ['home', 'house', 'mansion', 'smart houses', 'housing']
Cluster 225: ['irregular migration', 'migrant', 'refugee rights', 'refugee crisis', 'migration', 'migratory', 'refugee', 'migrant rights', 'migration and policy']
Cluster 4: ['conveyance', 'bringing up', 'carryover', 'bring', 'carry']
Cluster 184: ['progress', 'advancement', 'capability']
Cluster 0: ['voice', 'environmentally sound technologies', 'speech sound', 'soundly', 'auditory sensation', 'audio', 'sound', 'vocalize', 'vocalis

In [383]:
# Crear un diccionario para mapear cada cluster a una palabra clave representativa
cluster_representante = {}

for c, palabras in clusters_dict.items():
    if len(palabras) == 1:
        cluster_representante[c] = palabras[0]
    else:
        # Calcular la similitud promedio de cada palabra con las demás en el cluster
        indices = [palabras_clave_expandidas.index(p) for p in palabras]
        similitudes_cluster = cosine_similarity(embeddings_palabras_np[indices])
        sim_promedio = similitudes_cluster.mean(axis=1)
        # Seleccionar la palabra con la mayor similitud promedio
        representante_idx = np.argmax(sim_promedio)
        representante = palabras[representante_idx]
        cluster_representante[c] = representante

print("\nPalabras clave representativas por cluster:")
for c, rep in cluster_representante.items():
    print(f"Cluster {c}: {rep}")



Palabras clave representativas por cluster:
Cluster 158: broken
Cluster 8: communication
Cluster 331: median
Cluster 222: polite
Cluster 117: developed
Cluster 125: small
Cluster 232: tourism
Cluster 259: house
Cluster 225: migration
Cluster 4: bring
Cluster 381: downcast
Cluster 184: advancement
Cluster 0: sound
Cluster 211: universe
Cluster 231: manage
Cluster 434: h2o
Cluster 121: investment
Cluster 124: aim
Cluster 334: defend
Cluster 145: reuse
Cluster 221: abide
Cluster 84: raptus
Cluster 26: equalize
Cluster 240: maverick
Cluster 109: world
Cluster 152: care
Cluster 416: pocketsized
Cluster 403: intergenerational
Cluster 196: displacement
Cluster 162: agriculture
Cluster 5: put on the line
Cluster 223: ache
Cluster 97: pull
Cluster 120: ship
Cluster 267: keister
Cluster 82: equal rights to economic resources
Cluster 103: organization
Cluster 377: menage
Cluster 343: agendum
Cluster 19: data
Cluster 324: body politic
Cluster 71: disengage
Cluster 198: recycle
Cluster 1: humanity

In [384]:
# Actualizar el mapa_concepto_palabra para usar palabras clave representativas
mapa_concepto_palabra_representante = {}

for concepto, palabras in mapa_concepto_palabra.items():
    representantes = set()
    for palabra in palabras:
        # Encontrar el cluster de la palabra
        cluster_id = next((c for c, p in clusters_dict.items() if palabra in p), None)
        if cluster_id is not None:
            representante = cluster_representante[cluster_id]
            representantes.add(representante)
    if representantes:
        mapa_concepto_palabra_representante[concepto] = list(representantes)

# Ejemplo de mapeo actualizado
print("\nEjemplo de mapeo de concepto a palabras clave representativas:")
for concepto, palabras in list(mapa_concepto_palabra_representante.items())[:5]:
    print(f"- {concepto}: {palabras}")



Ejemplo de mapeo de concepto a palabras clave representativas:
- the organization: ['organization']
- the world trade organization: ['trade']
- developing countries proportion: ['international support for developing countries']
- coordinated policies: ['policy']
- many developed countries: ['international support for developing countries']


In [385]:
# Crear una lista para almacenar las palabras clave representativas por target
palabras_clave_por_target = []

for conceptos in tqdm(targets_conceptos, desc="Asignando palabras clave a targets"):
    palabras_asignadas = set()
    for concepto in conceptos:
        palabras = mapa_concepto_palabra_representante.get(concepto, [])
        palabras_asignadas.update(palabras)
    palabras_clave_por_target.append(list(palabras_asignadas))

# Crear un DataFrame para almacenar los resultados


# Crear un DataFrame para almacenar los resultados con goal_id y target_id
targets_df = pd.DataFrame({
    'goal_id': goal_ids,
    'target_id': target_ids,
    'target': targets,
    'target_normalizado': targets_normalizados,
    'conceptos': targets_conceptos,
    'palabras_clave': palabras_clave_por_target
})

# Mostrar un ejemplo
print("\nEjemplo de palabras clave asignadas a un target:")
print(targets_df[['target', 'palabras_clave']].iloc[0])


Asignando palabras clave a targets: 100%|██████████| 19/19 [00:00<?, ?it/s]


Ejemplo de palabras clave asignadas a un target:
target            Strengthen domestic resource mobilization, inc...
palabras_clave    [international support for developing countrie...
Name: 0, dtype: object


In [386]:
# Identificar targets sin palabras clave asignadas
targets_df['tiene_palabras_clave'] = targets_df['palabras_clave'].apply(lambda x: len(x) > 0)
targets_sin_palabras = targets_df[targets_df['tiene_palabras_clave'] == False]

print(f"\nTotal de targets sin palabras clave asignadas: {len(targets_sin_palabras)}")
print(targets_sin_palabras[['target', 'palabras_clave']])



Total de targets sin palabras clave asignadas: 0
Empty DataFrame
Columns: [target, palabras_clave]
Index: []


In [387]:
from collections import Counter

# Calcular la frecuencia de cada palabra clave
contador_palabras = Counter([palabra for sublist in palabras_clave_por_target for palabra in sublist])
palabras_frecuentes = [palabra for palabra, _ in contador_palabras.most_common(3)]  # Top 3 palabras clave

print("\nPalabras clave más frecuentes:", palabras_frecuentes)

# Asignar las top 3 palabras clave más frecuentes a targets sin palabras clave
for i, row in targets_df.iterrows():
    if not row['tiene_palabras_clave']:
        targets_df.at[i, 'palabras_clave'] = palabras_frecuentes

# Verificar nuevamente
print(f"\nTotal de targets sin palabras clave asignadas después de la asignación de palabras frecuentes: {len(targets_df[targets_df['palabras_clave'].apply(lambda x: len(x) == 0)])}")



Palabras clave más frecuentes: ['international support for developing countries', 'global partnership', 'environmental sustainability']

Total de targets sin palabras clave asignadas después de la asignación de palabras frecuentes: 0


In [388]:
# targets_df

In [389]:
import os

# Concatenar las palabras clave separadas por comas
targets_df['palabras_clave_concatenadas'] = targets_df['palabras_clave'].apply(lambda x: ', '.join(x))

# Seleccionar las columnas deseadas, incluyendo goal_id y target_id
resultado_csv = targets_df[['goal_id', 'target_id', 'target', 'palabras_clave_concatenadas']]

# Definir la ruta del archivo CSV final
file_path = 'targets_con_palabras_clave_agrupadas.csv'

# Verificar si el archivo ya existe
file_exists = os.path.isfile(file_path)

# Si el archivo ya existe, leerlo para evitar duplicados
if file_exists:
    final_df = pd.read_csv(file_path)
    # Filtrar los targets que ya están en el archivo final
    nuevos_resultados = resultado_csv[~resultado_csv['target_id'].isin(final_df['target_id'])]
else:
    nuevos_resultados = resultado_csv

# Guardar los nuevos resultados en el archivo CSV usando append si ya existe
nuevos_resultados.to_csv(file_path, mode='a', index=False, header=not file_exists)

print(f"\nArchivo CSV generado exitosamente: '{file_path}'")


Archivo CSV generado exitosamente: 'targets_con_palabras_clave_agrupadas.csv'
